重复使用的时候，只需要改变文档路径就可以了，关键参数分别是：
- root_sys: 对应系统的译文所在目录（因为使用多个系统，每个系统对应一个文件）
- root_score: 对应分数文件所在的目录，和系统译文一样，每个系统对应一个文件。
- dir_data_ref: 对应参考译文文件
- dir_data_scorece: 对应的原文
- sys: 所有使用的系统的名字
- 文件最后保存在“./data/”文件夹下
- 最后的乱序是没有必要的

另外应该注意的是，分数文件是有要求的，他应该有两列数据，第一列是sid，第二列是score。表示第sid个句子的译文的分数是score。如果没有现成的文件，这部分在observe_data2中实现。另外应该注意的是这些文件是必须按照sid进行排序的，也就是说，如果得到的是没有排序的文件是不能用的。<br>
重复运行的时候，一次性执行到写文件就行了。如果有时间会把他们集合到一个python文件里面的。

In [1]:
import os

In [27]:
# sys output是目录路径，因为有多个系统，其他两个是文件路径
root_sys = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_a_source_en.de/en-de/"
dir_data_ref = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_a_source_en.de/newstest2017-ende-ref.de"
dir_data_source = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_a_source_en.de/newstest2017-ende-src.en"
root_score = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_a_source_en.de/raw_scores/"

In [28]:
# 系统汇总 en-de
sys = ['C-3MA.4959','fbk-nmt-combination.4870', 'KIT.4950', 'LIUM-NMT.4900', 'LMU-nmt-reranked.4934', 'LMU-nmt-single.4893', 'online-A.0', 'online-B.0', 'online-F.0', 'online-G.0', 'PROMT-Rule-based.4735', 'RWTH-nmt-ensemble.4921', 'SYSTRAN.4847', 'TALP-UPC.4834', 'uedin-nmt.4722', 'xmu.4910']

In [16]:
# 系统汇总 de-en
sys = ['C-3MA.4958',
       'KIT.4951',
       'LIUM-NMT.4733',
       'online-A.0',
       'online-B.0',
       'online-F.0',
       'online-G.0',
       'RWTH-nmt-ensemble.4920',
       'SYSTRAN.4846',
       'TALP-UPC.4830',
       'uedin-nmt.4723']

In [29]:
sys

['C-3MA.4959',
 'fbk-nmt-combination.4870',
 'KIT.4950',
 'LIUM-NMT.4900',
 'LMU-nmt-reranked.4934',
 'LMU-nmt-single.4893',
 'online-A.0',
 'online-B.0',
 'online-F.0',
 'online-G.0',
 'PROMT-Rule-based.4735',
 'RWTH-nmt-ensemble.4921',
 'SYSTRAN.4847',
 'TALP-UPC.4834',
 'uedin-nmt.4722',
 'xmu.4910']

In [30]:
# 获取sys_out文件目录下的文件名
walk = os.walk(root_sys)
filenames = ""
for root, dir, fname in walk:
    filenames = fname
filenames

['newstest2017.online-F.0.en-de',
 'newstest2017.LIUM-NMT.4900.en-de',
 'newstest2017.LMU-nmt-reranked.4934.en-de',
 'newstest2017.online-G.0.en-de',
 'newstest2017.C-3MA.4959.en-de',
 'newstest2017.fbk-nmt-combination.4870.en-de',
 'newstest2017.RWTH-nmt-ensemble.4921.en-de',
 'newstest2017.PROMT-Rule-based.4735.en-de',
 'newstest2017.uedin-nmt.4722.en-de',
 'newstest2017.KIT.4950.en-de',
 'newstest2017.online-A.0.en-de',
 'newstest2017.xmu.4910.en-de',
 'newstest2017.SYSTRAN.4847.en-de',
 'newstest2017.online-B.0.en-de',
 'newstest2017.LMU-nmt-single.4893.en-de',
 'newstest2017.TALP-UPC.4834.en-de']

In [31]:
def add_one(file):
    tmp = open(file)
    for line in tmp:
        yield line
    yield None

In [32]:
def get_filename(name, filelist):
    index = 0
    while(index < len(filelist)):
        if name in filelist[index]:
            return filelist[index]
        index = index + 1
    if index == len(filelist):
        print("file not exist!!")
        return None

In [33]:
data_sys_out = []
data_ref = []
data_source = []
data_scores = []

In [34]:
def add_data(li_r, li_s, li_o, score):
    data_ref.append(li_r)
    data_source.append(li_s)
    data_sys_out.append(li_o)
    data_scores.append(score)

In [35]:
# 读文件

for sysname in sys:
    # 读取ref和src文件
    file_ref = add_one(dir_data_ref)
    file_src = add_one(dir_data_source)
    # 读取对应的sys_out文件，要求是名字包含有sysname的对应的文件
    filename = get_filename(sysname, filenames)
    file_sys_out = add_one(root_sys + "/" + filename)
    cfile = zip(file_ref, file_src, file_sys_out)
    
    # 读取analyse文件，并把行数和分数分别存在列表sids和tmp_scores中
    file_ana = open(root_score + sysname)
    sids = []
    tmp_scores = []
    for line in file_ana:
        tmp = line.split(",")
        sids.append(tmp[0])
        tmp_scores.append(tmp[1])
    
    assert(len(sids) == len(tmp_scores))
    print("start extract data from file " + "./test_data/" + sysname)
    print(sysname + " file long:", len(sids))
    index_sent = 1 # 指明当前看的是文档中的第几个句子，从1开始计数
    index_ana = 0 # 指明当前看的是ana文件中的第几项，从0开始计数
    end = False
    for item in cfile:
        # 因为文件是按照sid进行排序的，所以可以这么处理
        li_r = item[0]
        li_s = item[1]
        li_o = item[2]
        # 如果sid该项所看的句子，刚好是现在所在的句子，就把这个句子记起来，否则跳过
        # 可能存在一个句子有多个人打分的情况，所以这里用while，而不是用if
        # 因为不存在无用的项，所以仅在保存了该项的内容后，才会增加index_ana
        sid = sids[index_ana]
        while str(index_sent) == sid: 
            add_data(li_r, li_s, li_o, tmp_scores[index_ana])
            index_ana = index_ana + 1
            if index_ana < len(sids):
                sid = sids[index_ana]
            else:
                # 该系统的分析文件已经读完了，所以就结束程序吧
                end = True
                break
        # 假如index_sent和sid不一致，说明ana当前项看的并不是这个句子，所以跳到下一个句子
        index_sent = index_sent + 1
        if end:
            break
        
    print("..end extract data from file " + "./test_data/" + sysname)
    
    file_ref.close()
    file_src.close()
    file_sys_out.close()
    file_ana.close()

start extract data from file ./test_data/C-3MA.4959
('C-3MA.4959 file long:', 628)
..end extract data from file ./test_data/C-3MA.4959
start extract data from file ./test_data/fbk-nmt-combination.4870
('fbk-nmt-combination.4870 file long:', 525)
..end extract data from file ./test_data/fbk-nmt-combination.4870
start extract data from file ./test_data/KIT.4950
('KIT.4950 file long:', 532)
..end extract data from file ./test_data/KIT.4950
start extract data from file ./test_data/LIUM-NMT.4900
('LIUM-NMT.4900 file long:', 546)
..end extract data from file ./test_data/LIUM-NMT.4900
start extract data from file ./test_data/LMU-nmt-reranked.4934
('LMU-nmt-reranked.4934 file long:', 600)
..end extract data from file ./test_data/LMU-nmt-reranked.4934
start extract data from file ./test_data/LMU-nmt-single.4893
('LMU-nmt-single.4893 file long:', 477)
..end extract data from file ./test_data/LMU-nmt-single.4893
start extract data from file ./test_data/online-A.0
('online-A.0 file long:', 585)
..

In [36]:
# 写文件
def write_data(filename, li):
    with open(filename, 'w') as fi:
        for line in li:
            fi.write(line)

In [37]:
write_data("../data/MasterArbeit/plan_a_source_en.de/data_sys_out", data_sys_out)
write_data("../data/MasterArbeit/plan_a_source_en.de/data_ref", data_ref)
write_data("../data/MasterArbeit/plan_a_source_en.de/data_source", data_source)
write_data("../data/MasterArbeit/plan_a_source_en.de/data_scores", data_scores)

In [38]:
len(data_scores)

9392

In [12]:
# 把数据分成三部分：training data, validation data, pseudo test data
import random

In [21]:
# 乱序shuffle
random.seed(10)
random.shuffle(data_sys_out)
random.seed(10)
random.shuffle(data_ref)
random.seed(10)
random.shuffle(data_source)
random.seed(10)
random.shuffle(data_scores)

In [33]:
train_sys_out = []
train_ref = []
train_source = []
train_scores = []

val_sys_out = []
val_ref = []
val_source = []
val_scores = []

pseudo_test_sys_out = []
pseudo_test_ref = []
pseudo_test_source = []
pseudo_test_scores = []

In [43]:
sep1 = round(len(data_sys_out)*0.8)
sep2 = round(len(data_sys_out)*0.9)
end = len(data_sys_out) - 1

In [34]:
train_sys_out.extend(data_sys_out[0:sep1])
train_ref.extend(data_ref[0:sep1])
train_source.extend(data_source[0:sep1])
train_scores.extend(data_scores[0:sep1])

In [37]:
val_sys_out.extend(data_sys_out[sep1+1:sep2])
val_ref.extend(data_ref[sep1+1:sep2])
val_source.extend(data_source[sep1+1:sep2])
val_scores.extend(data_scores[sep1+1:sep2])

In [45]:
pseudo_test_sys_out.extend(data_sys_out[sep2+1:end])
pseudo_test_ref.extend(data_ref[sep2+1:end])
pseudo_test_source.extend(data_source[sep2+1:end])
pseudo_test_scores.extend(data_scores[sep2+1:end])

In [53]:
write_data("./data/train_sys_out", train_sys_out)
write_data("./data/train_ref", train_ref)
write_data("./data/train_source", train_source)
write_data("./data/train_scores", train_scores)

write_data("./data/val_sys_out", val_sys_out)
write_data("./data/val_ref", val_ref)
write_data("./data/val_source", val_source)
write_data("./data/val_scores", val_scores)

write_data("./data/pseudo_test_sys_out", pseudo_test_sys_out)
write_data("./data/pseudo_test_ref", pseudo_test_ref)
write_data("./data/pseudo_test_source", pseudo_test_source)
write_data("./data/pseudo_test_scores", pseudo_test_scores)

### plan B 句子提取

In [39]:
import os

In [61]:
# sys output是目录路径，因为有多个系统，其他两个是文件路径
root_sys = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_b_source_de.en/de-en/"
dir_data_ref = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_b_source_de.en/newstest2017-deen-ref.en"
dir_data_source = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_b_source_de.en/newstest2017-deen-src.de"
root_score = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_b_source_de.en/raw_scores/"

In [43]:
# 系统汇总 en-de
sys = ['C-3MA.4959','fbk-nmt-combination.4870', 'KIT.4950', 'LIUM-NMT.4900', 'LMU-nmt-reranked.4934', 'LMU-nmt-single.4893', 'online-A.0', 'online-B.0', 'online-F.0', 'online-G.0', 'PROMT-Rule-based.4735', 'RWTH-nmt-ensemble.4921', 'SYSTRAN.4847', 'TALP-UPC.4834', 'uedin-nmt.4722', 'xmu.4910']

In [62]:
# 系统汇总 de-en
sys = ['C-3MA.4958',
       'KIT.4951',
       'LIUM-NMT.4733',
       'online-A.0',
       'online-B.0',
       'online-F.0',
       'online-G.0',
       'RWTH-nmt-ensemble.4920',
       'SYSTRAN.4846',
       'TALP-UPC.4830',
       'uedin-nmt.4723']

In [44]:
sys

['C-3MA.4959',
 'fbk-nmt-combination.4870',
 'KIT.4950',
 'LIUM-NMT.4900',
 'LMU-nmt-reranked.4934',
 'LMU-nmt-single.4893',
 'online-A.0',
 'online-B.0',
 'online-F.0',
 'online-G.0',
 'PROMT-Rule-based.4735',
 'RWTH-nmt-ensemble.4921',
 'SYSTRAN.4847',
 'TALP-UPC.4834',
 'uedin-nmt.4722',
 'xmu.4910']

In [63]:
# 获取sys_out文件目录下的文件名
walk = os.walk(root_sys)
filenames = ""
for root, dir, fname in walk:
    filenames = fname
filenames

['newstest2017.C-3MA.4958.de-en',
 'newstest2017.online-F.0.de-en',
 'newstest2017.RWTH-nmt-ensemble.4920.de-en',
 'newstest2017.online-G.0.de-en',
 'newstest2017.LIUM-NMT.4733.de-en',
 'newstest2017.online-A.0.de-en',
 'newstest2017.uedin-nmt.4723.de-en',
 'newstest2017.KIT.4951.de-en',
 'newstest2017.online-B.0.de-en',
 'newstest2017.TALP-UPC.4830.de-en',
 'newstest2017.SYSTRAN.4846.de-en']

In [64]:
def add_one(file):
    tmp = open(file)
    for line in tmp:
        yield line
    yield None

In [65]:
data_sys_out = []
data_ref = []
data_source = []
data_scores = []

In [66]:
def add_data(li_r, li_s, li_o, score):
    data_ref.append(li_r)
    data_source.append(li_s)
    data_sys_out.append(li_o)
    data_scores.append(score)

下面的方法是以此提取sys中的文件，而我想要的是按照sys进行分类，所以这里我只需要把划分这里的sys就可以了。<br>
应该注意的是，之后的保存的时候，不能把名字弄混了

In [77]:
sys = ['uedin-nmt.4723']

In [78]:
for sysname in sys:
    # 读取ref和src文件
    file_ref = add_one(dir_data_ref)
    file_src = add_one(dir_data_source)
    # 读取对应的sys_out文件，要求是名字包含有sysname的对应的文件
    filename = get_filename(sysname, filenames)
    file_sys_out = add_one(root_sys + "/" + filename)
    cfile = zip(file_ref, file_src, file_sys_out)
    
    # 读取analyse文件，并把行数和分数分别存在列表sids和tmp_scores中
    file_ana = open(root_score + sysname)
    sids = []
    tmp_scores = []
    for line in file_ana:
        tmp = line.split(",")
        sids.append(tmp[0])
        tmp_scores.append(tmp[1])
    
    assert(len(sids) == len(tmp_scores))
    print("start extract data from file " + "./test_data/" + sysname)
    print(sysname + " file long:", len(sids))
    index_sent = 1 # 指明当前看的是文档中的第几个句子，从1开始计数
    index_ana = 0 # 指明当前看的是ana文件中的第几项，从0开始计数
    end = False
    for item in cfile:
        # 因为文件是按照sid进行排序的，所以可以这么处理
        li_r = item[0]
        li_s = item[1]
        li_o = item[2]
        # 如果sid该项所看的句子，刚好是现在所在的句子，就把这个句子记起来，否则跳过
        # 可能存在一个句子有多个人打分的情况，所以这里用while，而不是用if
        # 因为不存在无用的项，所以仅在保存了该项的内容后，才会增加index_ana
        sid = sids[index_ana]
        while str(index_sent) == sid: 
            add_data(li_r, li_s, li_o, tmp_scores[index_ana])
            index_ana = index_ana + 1
            if index_ana < len(sids):
                sid = sids[index_ana]
            else:
                # 该系统的分析文件已经读完了，所以就结束程序吧
                end = True
                break
        # 假如index_sent和sid不一致，说明ana当前项看的并不是这个句子，所以跳到下一个句子
        index_sent = index_sent + 1
        if end:
            break
        
    print("..end extract data from file " + "./test_data/" + sysname)
    
    file_ref.close()
    file_src.close()
    file_sys_out.close()
    file_ana.close()

start extract data from file ./test_data/uedin-nmt.4723
('uedin-nmt.4723 file long:', 2185)
..end extract data from file ./test_data/uedin-nmt.4723


In [79]:
# 写文件
def write_data(filename, li):
    with open(filename, 'w') as fi:
        for line in li:
            fi.write(line)

In [80]:
write_data("../data/MasterArbeit/plan_b_source_de.en/test_sys_out", data_sys_out)
write_data("../data/MasterArbeit/plan_b_source_de.en/test_ref", data_ref)
write_data("../data/MasterArbeit/plan_b_source_de.en/test_source", data_source)
write_data("../data/MasterArbeit/plan_b_source_de.en/test_scores", data_scores)

## Plan C

In [1]:
import os

In [73]:
root_sys = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_c_source_rank_de.en/de-en_2014"
dir_src = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_c_source_rank_de.en/newstest2014-deen-src.de"
dir_ref = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_c_source_rank_de.en/newstest2014-deen-ref.en"
dir_result = "/Users/ihuangyiran/Documents/Workplace_Python/data/MasterArbeit/plan_c_source_rank_de.en/result_2014"

In [74]:
walk = os.walk(root_sys)
filenames = ""
for root, dir, fname in walk:
    filenames = fname
filenames

['newstest2014.LIMSI-KIT-Submission.3359.de-en',
 'newstest2014.onlineA.0.de-en',
 'newstest2014.eubridge.3569.de-en',
 'newstest2014.rbmt1.0.de-en',
 'newstest2014.CMU.3461.de-en',
 'newstest2014.onlineB.0.de-en',
 'newstest2014.uedin-wmt14.3025.de-en',
 'newstest2014.onlineC.0.de-en',
 'newstest2014.rbmt4.0.de-en',
 'newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en',
 'newstest2014.kit.3109.de-en',
 'newstest2014.uedin-syntax.3035.de-en',
 'newstest2014.RWTH-primary.3266.de-en']

In [75]:
# load the file to a list
def add_one(file):
    con = []
    tmp = open(file)
    for line in tmp:
        con.append(line)
    return con

In [76]:
# combine all the contents of the file with a dict
dict_sys = {}
for filename in filenames:
    dict_sys[filename] = add_one(root_sys + '/' + filename)


In [77]:
# load the content in source file and ref file
con_src = add_one(dir_src)
con_ref = add_one(dir_ref)

In [78]:
# define the function to add the content
def add_data(src, ref, s1, s2, result):
    data_src.append(src)
    data_ref.append(ref)
    data_s1.append(s1)
    data_s2.append(s2)
    data_result.append(result)

In [79]:
# set the attribute to hold the data
data_src = []
data_ref = []
data_s1 = []
data_s2 = []
data_result = []

In [80]:
with open(dir_result) as fi:
    for line in fi:
        tmp = line.split(',')
        # cols of tmp: 'sid', 's1', 's2', 'result'
        # pay attention to that, the sid begin with 1 but the index is zero 
        sid = int(tmp[0].strip())
        name_s1 = tmp[1][1: -1]
        name_s2 = tmp[2][1: -1]
        sen_src = con_src[sid - 1]
        sen_ref = con_ref[sid - 1]
        sen_s1 = dict_sys[name_s1][sid - 1]
        sen_s2 = dict_sys[name_s2][sid - 1]
        sen_result = tmp[3]
        add_data(sen_src, sen_ref, sen_s1, sen_s2, sen_result)

In [81]:
# 写文件
def write_data(filename, li):
    with open(filename, 'w') as fi:
        for line in li:
            fi.write(line)

In [82]:
with open(dir_result) as fi:
    for line in fi:
        print line

1,"newstest2014.onlineA.0.de-en","newstest2014.rbmt1.0.de-en",-1

1,"newstest2014.onlineA.0.de-en","newstest2014.onlineC.0.de-en",-1

1,"newstest2014.onlineA.0.de-en","newstest2014.uedin-syntax.3035.de-en",0

1,"newstest2014.onlineA.0.de-en","newstest2014.eubridge.3569.de-en",0

1,"newstest2014.rbmt1.0.de-en","newstest2014.onlineC.0.de-en",0

1,"newstest2014.rbmt1.0.de-en","newstest2014.uedin-syntax.3035.de-en",1

1,"newstest2014.rbmt1.0.de-en","newstest2014.eubridge.3569.de-en",1

1,"newstest2014.onlineC.0.de-en","newstest2014.uedin-syntax.3035.de-en",1

1,"newstest2014.onlineC.0.de-en","newstest2014.eubridge.3569.de-en",1

1,"newstest2014.uedin-syntax.3035.de-en","newstest2014.eubridge.3569.de-en",0

3,"newstest2014.onlineA.0.de-en","newstest2014.rbmt1.0.de-en",-1

3,"newstest2014.onlineA.0.de-en","newstest2014.onlineC.0.de-en",-1

3,"newstest2014.onlineA.0.de-en","newstest2014.uedin-syntax.3035.de-en",-1

3,"newstest2014.onlineA.0.de-en","newstest2014.eubridge.3569.de-en",-1

3,"new

72,"newstest2014.onlineC.0.de-en","newstest2014.eubridge.3569.de-en",1

72,"newstest2014.rbmt4.0.de-en","newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en",1

72,"newstest2014.RWTH-primary.3266.de-en","newstest2014.rbmt4.0.de-en",1

72,"newstest2014.rbmt4.0.de-en","newstest2014.onlineB.0.de-en",-1

72,"newstest2014.RWTH-primary.3266.de-en","newstest2014.rbmt1.0.de-en",0

72,"newstest2014.rbmt4.0.de-en","newstest2014.eubridge.3569.de-en",1

72,"newstest2014.RWTH-primary.3266.de-en","newstest2014.eubridge.3569.de-en",1

72,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en","newstest2014.onlineB.0.de-en",-1

72,"newstest2014.rbmt4.0.de-en","newstest2014.rbmt1.0.de-en",-1

72,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en","newstest2014.eubridge.3569.de-en",0

72,"newstest2014.rbmt4.0.de-en","newstest2014.eubridge.3569.de-en",1

72,"newstest2014.onlineB.0.de-en","newstest2014.eubridge.3569.de-en",1

72,"newstest2014.rbmt1.0.de-en","newstest2014.eubridge.3569.de-en",1

73,"newstest2014.LIMSI-KIT-S


220,"newstest2014.onlineA.0.de-en","newstest2014.uedin-wmt14.3025.de-en",1

220,"newstest2014.onlineA.0.de-en","newstest2014.onlineB.0.de-en",1

220,"newstest2014.onlineA.0.de-en","newstest2014.onlineC.0.de-en",0

220,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.onlineB.0.de-en",0

220,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.onlineC.0.de-en",-1

220,"newstest2014.onlineB.0.de-en","newstest2014.onlineC.0.de-en",-1

222,"newstest2014.rbmt1.0.de-en","newstest2014.onlineA.0.de-en",-1

222,"newstest2014.rbmt1.0.de-en","newstest2014.uedin-wmt14.3025.de-en",-1

222,"newstest2014.rbmt1.0.de-en","newstest2014.onlineB.0.de-en",-1

222,"newstest2014.rbmt1.0.de-en","newstest2014.onlineC.0.de-en",-1

222,"newstest2014.onlineA.0.de-en","newstest2014.uedin-wmt14.3025.de-en",-1

222,"newstest2014.onlineA.0.de-en","newstest2014.onlineB.0.de-en",0

222,"newstest2014.onlineA.0.de-en","newstest2014.onlineC.0.de-en",-1

222,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.onlineB.0.d


354,"newstest2014.rbmt4.0.de-en","newstest2014.rbmt1.0.de-en",-1

354,"newstest2014.CMU.3461.de-en","newstest2014.uedin-syntax.3035.de-en",-1

354,"newstest2014.CMU.3461.de-en","newstest2014.uedin-syntax.3035.de-en",1

354,"newstest2014.onlineA.0.de-en","newstest2014.uedin-syntax.3035.de-en",-1

354,"newstest2014.onlineC.0.de-en","newstest2014.LIMSI-KIT-Submission.3359.de-en",1

354,"newstest2014.CMU.3461.de-en","newstest2014.eubridge.3569.de-en",1

354,"newstest2014.CMU.3461.de-en","newstest2014.eubridge.3569.de-en",1

354,"newstest2014.onlineA.0.de-en","newstest2014.kit.3109.de-en",-1

354,"newstest2014.onlineC.0.de-en","newstest2014.rbmt1.0.de-en",0

354,"newstest2014.uedin-syntax.3035.de-en","newstest2014.eubridge.3569.de-en",1

354,"newstest2014.uedin-syntax.3035.de-en","newstest2014.eubridge.3569.de-en",1

354,"newstest2014.uedin-syntax.3035.de-en","newstest2014.kit.3109.de-en",-1

354,"newstest2014.LIMSI-KIT-Submission.3359.de-en","newstest2014.rbmt1.0.de-en",-1

362,"newstest2

537,"newstest2014.RWTH-primary.3266.de-en","newstest2014.onlineC.0.de-en",1

537,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en","newstest2014.rbmt4.0.de-en",1

537,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en","newstest2014.onlineC.0.de-en",0

537,"newstest2014.rbmt4.0.de-en","newstest2014.onlineC.0.de-en",-1

541,"newstest2014.CMU.3461.de-en","newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en",-1

541,"newstest2014.CMU.3461.de-en","newstest2014.uedin-wmt14.3025.de-en",-1

541,"newstest2014.CMU.3461.de-en","newstest2014.eubridge.3569.de-en",-1

541,"newstest2014.CMU.3461.de-en","newstest2014.onlineC.0.de-en",0

541,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en","newstest2014.uedin-wmt14.3025.de-en",0

541,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en","newstest2014.eubridge.3569.de-en",0

541,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en","newstest2014.onlineC.0.de-en",1

541,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.eubridge.3569.de-en",0

541,"newstest2014.uedin-wmt14.302

...

2192,"newstest2014.onlineC.0.de-en","newstest2014.uedin-wmt14.3025.de-en",-1

2192,"newstest2014.onlineC.0.de-en","newstest2014.uedin-wmt14.3025.de-en",-1

2192,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.CMU.3461.de-en",0

2193,"newstest2014.onlineA.0.de-en","newstest2014.uedin-syntax.3035.de-en",1

2193,"newstest2014.onlineA.0.de-en","newstest2014.uedin-syntax.3035.de-en",0

2193,"newstest2014.onlineC.0.de-en","newstest2014.eubridge.3569.de-en",1

2193,"newstest2014.onlineA.0.de-en","newstest2014.eubridge.3569.de-en",1

2193,"newstest2014.onlineA.0.de-en","newstest2014.eubridge.3569.de-en",-1

2193,"newstest2014.onlineC.0.de-en","newstest2014.LIMSI-KIT-Submission.3359.de-en",1

2193,"newstest2014.onlineA.0.de-en","newstest2014.onlineC.0.de-en",1

2193,"newstest2014.onlineA.0.de-en","newstest2014.onlineC.0.de-en",0

2193,"newstest2014.onlineC.0.de-en","newstest2014.uedin-wmt14.3025.de-en",1

2193,"newstest2014.onlineA.0.de-en","newstest2014.uedin-wmt14.3025.de-en",1

2193,"new


2348,"newstest2014.rbmt1.0.de-en","newstest2014.onlineB.0.de-en",0

2348,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en","newstest2014.RWTH-primary.3266.de-en",0

2348,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en","newstest2014.onlineB.0.de-en",0

2348,"newstest2014.RWTH-primary.3266.de-en","newstest2014.onlineB.0.de-en",0

2349,"newstest2014.kit.3109.de-en","newstest2014.rbmt1.0.de-en",-1

2349,"newstest2014.kit.3109.de-en","newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en",0

2349,"newstest2014.kit.3109.de-en","newstest2014.RWTH-primary.3266.de-en",0

2349,"newstest2014.kit.3109.de-en","newstest2014.onlineB.0.de-en",0

2349,"newstest2014.rbmt1.0.de-en","newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en",1

2349,"newstest2014.rbmt1.0.de-en","newstest2014.RWTH-primary.3266.de-en",1

2349,"newstest2014.rbmt1.0.de-en","newstest2014.onlineB.0.de-en",1

2349,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en","newstest2014.RWTH-primary.3266.de-en",0

2349,"newstest2014.DCU-ICTCAS-Tsinghua-L.3444.d

2496,"newstest2014.onlineB.0.de-en","newstest2014.CMU.3461.de-en",-1

2496,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.rbmt1.0.de-en",-1

2496,"newstest2014.onlineB.0.de-en","newstest2014.uedin-wmt14.3025.de-en",-1

2496,"newstest2014.LIMSI-KIT-Submission.3359.de-en","newstest2014.rbmt1.0.de-en",-1

2496,"newstest2014.CMU.3461.de-en","newstest2014.uedin-wmt14.3025.de-en",0

2501,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.onlineB.0.de-en",-1

2501,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.LIMSI-KIT-Submission.3359.de-en",-1

2501,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.kit.3109.de-en",-1

2501,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.eubridge.3569.de-en",-1

2501,"newstest2014.onlineB.0.de-en","newstest2014.LIMSI-KIT-Submission.3359.de-en",1

2501,"newstest2014.onlineB.0.de-en","newstest2014.kit.3109.de-en",-1

2501,"newstest2014.onlineB.0.de-en","newstest2014.eubridge.3569.de-en",-1

2501,"newstest2014.LIMSI-KIT-Submission.3359.de-en","


2672,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.LIMSI-KIT-Submission.3359.de-en",1

2672,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.eubridge.3569.de-en",1

2672,"newstest2014.LIMSI-KIT-Submission.3359.de-en","newstest2014.eubridge.3569.de-en",0

2673,"newstest2014.kit.3109.de-en","newstest2014.uedin-syntax.3035.de-en",-1

2673,"newstest2014.kit.3109.de-en","newstest2014.uedin-wmt14.3025.de-en",1

2673,"newstest2014.kit.3109.de-en","newstest2014.LIMSI-KIT-Submission.3359.de-en",1

2673,"newstest2014.kit.3109.de-en","newstest2014.eubridge.3569.de-en",1

2673,"newstest2014.uedin-syntax.3035.de-en","newstest2014.uedin-wmt14.3025.de-en",1

2673,"newstest2014.uedin-syntax.3035.de-en","newstest2014.LIMSI-KIT-Submission.3359.de-en",1

2673,"newstest2014.uedin-syntax.3035.de-en","newstest2014.eubridge.3569.de-en",1

2673,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.LIMSI-KIT-Submission.3359.de-en",0

2673,"newstest2014.uedin-wmt14.3025.de-en","newstest2014.eubridge.356

2894,"newstest2014.kit.3109.de-en","newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en",0

2894,"newstest2014.onlineC.0.de-en","newstest2014.uedin-syntax.3035.de-en",-1

2894,"newstest2014.onlineC.0.de-en","newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en",-1

2894,"newstest2014.uedin-syntax.3035.de-en","newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en",0

2895,"newstest2014.eubridge.3569.de-en","newstest2014.kit.3109.de-en",-1

2895,"newstest2014.eubridge.3569.de-en","newstest2014.onlineC.0.de-en",-1

2895,"newstest2014.eubridge.3569.de-en","newstest2014.uedin-syntax.3035.de-en",0

2895,"newstest2014.eubridge.3569.de-en","newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en",-1

2895,"newstest2014.kit.3109.de-en","newstest2014.onlineC.0.de-en",0

2895,"newstest2014.kit.3109.de-en","newstest2014.uedin-syntax.3035.de-en",1

2895,"newstest2014.kit.3109.de-en","newstest2014.DCU-ICTCAS-Tsinghua-L.3444.de-en",0

2895,"newstest2014.onlineC.0.de-en","newstest2014.uedin-syntax.3035.de-en",1

2895,"newstest2014.

In [83]:
# write the data to local
write_data("../data/MasterArbeit/plan_c_source_rank_de.en/data_src_2014", data_src)
write_data("../data/MasterArbeit/plan_c_source_rank_de.en/data_ref_2014", data_ref)
write_data("../data/MasterArbeit/plan_c_source_rank_de.en/data_s1_2014", data_s1)
write_data("../data/MasterArbeit/plan_c_source_rank_de.en/data_s2_2014", data_s2)
write_data("../data/MasterArbeit/plan_c_source_rank_de.en/data_result_2014", data_result)

### validation


In [ ]:
fi = open